## **Blood Pressure Prediction**

This notebook aims to predict blood pressure using bioimpedance measurements from graphene electronic tattoos.

For more detail and access to the data read this [article](https://physicsworld.com/a/graphene-tattoo-provides-cuffless-blood-pressure-monitoring/).

### **1.Data Exploration and Preprocessing**

The goal of this section is to combine all of the CSVs into one pandas dataframe with the following schema:

**Subject, Setup, Trial, Time, Bioz, FinapresBP, FinapresPPG, PPG**

In [19]:
import pandas as pd
import os 
import glob 

Lets start by reading the data related to one patient and setup.

In [49]:
# The path to the setup results
path = "../../data/continuous-cuffless-monitoring-of-arterial-blood-pressure-via-graphene-bioimpedance-tattoos-1.0.0/subject1_day1/setup01_baseline"

csv_files = glob.glob(os.path.join(path, "*.csv")) 

# Initialise an empty dataframe
df = pd.DataFrame()

# Loop over the list of csv files 
for f in csv_files: 

    # To avoid issues caused by mixing Windows and Linux paths replace \\ with /
    f = f.replace("\\", "/")

    # Extract the trial, setup and subject info from the file path
    trial = f.split("/")[-1]
    setup = f.split("/")[-2]
    subject = f.split("/")[-3]

    # Read the csv file and add the ids used to identify the experiment
    next_df = pd.read_csv(f)
    next_df['Subject'] = subject
    next_df['Setup'] = setup
    next_df['Trial'] = trial.split('_')[1]  # Should be shared across files from the same trial

    df = pd.concat([df, next_df], ignore_index=True)

In [50]:
df

,time,BioZ1,BioZ2,BioZ3,BioZ4,Subject,Setup,Trial,FinapresBP,FinapresPPG,PPG
0,1.000021,45.11402,18.60437,20.92109,6.67912,subject1_day1,setup01_baseline,trial01,NaN,NaN,NaN
1,1.000821,45.19330,18.60317,20.84464,6.61070,subject1_day1,setup01_baseline,trial01,NaN,NaN,NaN
2,1.001621,45.27085,18.60096,20.76674,6.54201,subject1_day1,setup01_baseline,trial01,NaN,NaN,NaN
3,1.002421,45.34667,18.59775,20.68740,6.47308,subject1_day1,setup01_baseline,trial01,NaN,NaN,NaN
4,1.003221,45.42077,18.59353,20.60664,6.40395,subject1_day1,setup01_baseline,trial01,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...
479524,87.596021,NaN,NaN,NaN,NaN,subject1_day1,setup01_baseline,trial02,NaN,NaN,-0.13551
479525,87.596821,NaN,NaN,NaN,NaN,subject1_day1,setup01_baseline,trial02,NaN,NaN,-0.14933
479526,87.597621,NaN,NaN,NaN,NaN,subject1_day1,setup01_baseline,trial02,NaN,NaN,-0.16312
479527,87.598421,NaN,NaN,NaN,NaN,subject1_day1,setup01_baseline,trial02,NaN,NaN,-0.17685


Unfortunately, values from the same experiment at the same time are on different rows.

To resolve this Pivot the value data using melt()

In [56]:
df_melted = pd.melt(df, id_vars=['Subject', 'Setup', 'Trial', 'time'], var_name='Metric', value_name='Value')
df_melted

,Subject,Setup,Trial,time,Metric,Value
0,subject1_day1,setup01_baseline,trial01,1.000021,BioZ1,45.11402
1,subject1_day1,setup01_baseline,trial01,1.000821,BioZ1,45.19330
2,subject1_day1,setup01_baseline,trial01,1.001621,BioZ1,45.27085
3,subject1_day1,setup01_baseline,trial01,1.002421,BioZ1,45.34667
4,subject1_day1,setup01_baseline,trial01,1.003221,BioZ1,45.42077
...,...,...,...,...,...,...
3356698,subject1_day1,setup01_baseline,trial02,87.596021,PPG,-0.13551
3356699,subject1_day1,setup01_baseline,trial02,87.596821,PPG,-0.14933
3356700,subject1_day1,setup01_baseline,trial02,87.597621,PPG,-0.16312
3356701,subject1_day1,setup01_baseline,trial02,87.598421,PPG,-0.17685


Then Unpivot it back to have a column for each value.

Now Metrics from the same time and trial are on the same row.

In [63]:
df = df_melted.pivot_table(index=['Subject', 'Setup', 'Trial', 'time'], columns='Metric', values='Value').reset_index()
df

Metric,Subject,Setup,Trial,time,BioZ1,BioZ2,BioZ3,BioZ4,FinapresBP,FinapresPPG,PPG
0,subject1_day1,setup01_baseline,trial01,1.000021,45.11402,18.60437,20.92109,6.67912,NaN,NaN,12.85023
1,subject1_day1,setup01_baseline,trial01,1.000821,45.19330,18.60317,20.84464,6.61070,NaN,NaN,12.72152
2,subject1_day1,setup01_baseline,trial01,1.001621,45.27085,18.60096,20.76674,6.54201,NaN,NaN,12.59401
3,subject1_day1,setup01_baseline,trial01,1.002421,45.34667,18.59775,20.68740,6.47308,NaN,NaN,12.46770
4,subject1_day1,setup01_baseline,trial01,1.003221,45.42077,18.59353,20.60664,6.40395,NaN,NaN,12.34258
...,...,...,...,...,...,...,...,...,...,...,...
263024,subject1_day1,setup01_baseline,trial02,87.596021,35.90495,12.90030,8.22138,5.59188,NaN,NaN,-0.13551
263025,subject1_day1,setup01_baseline,trial02,87.596821,36.05002,12.96404,8.24659,5.66730,NaN,NaN,-0.14933
263026,subject1_day1,setup01_baseline,trial02,87.597621,36.19220,13.02675,8.27244,5.74181,NaN,NaN,-0.16312
263027,subject1_day1,setup01_baseline,trial02,87.598421,36.33149,13.08843,8.29893,5.81538,NaN,NaN,-0.17685
